# SPCBViTNet – Vgg16 Notebook

📌 Part of the submission titled:  
**"SPCBViTNet: Enhancing Skin Cancer Diagnosis with Multi-Scale Vision Transformers and Adaptive Spatial-Channel Attention"**

📝 This notebook supports reproducibility during peer review.  
🔒 Not yet peer-reviewed. Please do not cite or reuse until publication.

📁 See full repo: [https://github.com/diyagoyal31/SPCBViT](https://github.com/diyagoyal31/SPCBViT)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving dataset.zip to dataset.zip


In [ ]:
import zipfile
import os

zip_path = "/content/dataset.zip"  # Change this to your file name
extract_path = "/content/dataset"  # Change this if you want a different folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed. Files are in:", extract_path)

Extraction completed. Files are in: /content/dataset


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os

In [ ]:
# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class SkinCancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        self.class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.class_names)}

        for cls_name in self.class_names:
            cls_folder = os.path.join(root_dir, cls_name)
            if not os.path.isdir(cls_folder):  # ✅ Skip non-directory files (e.g., .DS_Store)
                continue
            for img_name in os.listdir(cls_folder):
                img_path = os.path.join(cls_folder, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):  # ✅ Ensure it's an image file
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")  # Convert grayscale images to RGB

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
import torchvision.transforms.functional as TF
import random

In [ ]:
# Define Custom Transform with Probability for Color Adjustments
class RandomColorJitter:
    def __init__(self, p=0.4):
        self.p = p
        self.color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)

    def __call__(self, img):
        if random.random() < self.p:
            img = self.color_jitter(img)
        return img

In [ ]:
def pad_to_224(image):
    """
    Pad the image to 224x224 if it's smaller than that.
    """
    width, height = image.size  # Get current image size
    if width < 224 or height < 224:
        padding_left = (224 - width) // 2
        padding_top = (224 - height) // 2
        padding_right = 224 - width - padding_left
        padding_bottom = 224 - height - padding_top
        image = transforms.Pad((padding_left, padding_top, padding_right, padding_bottom))(image)
    return image

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of flipping
    RandomColorJitter(p=0.4),  # 40% chance of adjusting brightness, contrast, saturation, hue
    transforms.Lambda(pad_to_224),
    transforms.RandomRotation(30),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# No augmentation for test data
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(pad_to_224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Define Paths
train_dir = "/content/dataset/dataset/train"
test_dir = "/content/dataset/dataset/test"

In [ ]:
train_dataset = SkinCancerDataset(root_dir=train_dir, transform=train_transform)
test_dataset = SkinCancerDataset(root_dir=test_dir, transform=test_transform)

In [ ]:
# Create Data Loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
# Get number of classes
num_classes = len(train_dataset.class_names)
print(f"Number of classes: {num_classes}")

Number of classes: 7


Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.empty_cache()


# Selective Kernel Feature Pyramid Module (SKNet)
class SKFeaturePyramid(nn.Module):
    def __init__(self, in_channels):
        super(SKFeaturePyramid, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels, in_channels, kernel_size=5, padding=2)
        self.fc = nn.Linear(in_channels, in_channels)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        f1 = self.conv1(x)
        f2 = self.conv3(x)
        f3 = self.conv5(x)

        f_sum = f1 + f2 + f3
        f_weights = self.softmax(self.fc(f_sum.mean(dim=(2, 3))))

        out = f1 * f_weights[:, 0].view(-1, 1, 1, 1) + \
              f2 * f_weights[:, 1].view(-1, 1, 1, 1) + \
              f3 * f_weights[:, 2].view(-1, 1, 1, 1)
        return out

# Self-Interactive Attention Pyramid (SAP)
class SelfInteractiveAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfInteractiveAttention, self).__init__()
        self.channel_attn = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attn = nn.Conv2d(in_channels, 1, kernel_size=7, padding=3)

    def forward(self, x):
        ca = self.channel_attn(x) * x  # Channel Attention
        sa = torch.sigmoid(self.spatial_attn(x)) * x  # Spatial Attention
        out = ca + sa
        return out

# Higher-Order Interaction Module
class HigherOrderInteraction(nn.Module):
    def __init__(self, in_channels1, in_channels2, out_channels):
        super(HigherOrderInteraction, self).__init__()
        self.conv1 = nn.Conv2d(in_channels1, out_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels2, out_channels, kernel_size=1)
        self.bilinear_pool = nn.Conv2d(out_channels, out_channels, kernel_size=1)
        self.trilinear_pool = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x1, x2):

        x1 = self.conv1(x1)
        x2 = self.conv2(x2)

        # Match spatial sizes
        if x1.shape[2:] != x2.shape[2:]:
            x2 = F.interpolate(x2, size=x1.shape[2:], mode='bilinear', align_corners=False)

        out = self.bilinear_pool(x1 * x2) + self.trilinear_pool(x1 + x2)
        return out

# SPCB-Net (VGG16 Backbone)
class SPCBNet(nn.Module):
    def __init__(self, num_classes):
        super(SPCBNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (64, 112, 112)

            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (128, 56, 56)

            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: (256, 28, 28)

            nn.Conv2d(256, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),nn.BatchNorm2d(512), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Output: (512, 14, 14)
        )

        self.sk_block1 = SKFeaturePyramid(64)
        self.sk_block2 = SKFeaturePyramid(128)
        self.sk_block3 = SKFeaturePyramid(256)
        self.sk_block4 = SKFeaturePyramid(512)

        self.attention1 = SelfInteractiveAttention(64)
        self.attention2 = SelfInteractiveAttention(128)
        self.attention3 = SelfInteractiveAttention(256)
        self.attention4 = SelfInteractiveAttention(512)

        self.higher_order1 = HigherOrderInteraction(64, 128, 128)
        self.higher_order2 = HigherOrderInteraction(256, 512, 512)

        self.global_avg = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(512, num_classes)

    def forward(self, x):
        x1 = self.features[:4](x)   # B1 (64, 112, 112)
        x2 = self.features[4:9](x1) # B2 (128, 56, 56)
        x3 = self.features[9:16](x2)# B3 (256, 28, 28)
        x4 = self.features[16:](x3) # B4 (512, 14, 14)

        f1 = self.sk_block1(x1)
        f2 = self.sk_block2(x2)
        f3 = self.sk_block3(x3)
        f4 = self.sk_block4(x4)

        a1 = self.attention1(f1)
        a2 = self.attention2(f2)
        a3 = self.attention3(f3)
        a4 = self.attention4(f4)

        # Higher-Order Interaction
        ho1 = self.higher_order1(a1, a2)  # (2, 128, 224, 224)
        ho2 = self.higher_order2(a3, a4)  # (2, 512, 56, 56)

        # Resize ho1 spatially to match ho2
        ho1_resized = F.interpolate(ho1, size=ho2.shape[2:], mode='bilinear', align_corners=False)  # (2, 128, 56, 56)

        # Use 1x1 convolution to match channel size
        conv1x1 = nn.Conv2d(128, 512, kernel_size=1, stride=1, bias=False).to(ho1.device)
        ho1_resized = conv1x1(ho1_resized)  # (2, 512, 56, 56)

        # Global Average Pooling
        out = self.global_avg(ho1_resized + ho2).view(x.size(0), -1)
        out = self.fc1(out)



        return out

# Test Run
model = SPCBNet(num_classes=7)
x = torch.randn(2, 3, 224, 224)
output = model(x)

Training

In [ ]:
from tqdm import tqdm
import torch

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Training
    for epoch in range(num_epochs):
        total_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()  # ✅ Use GradScaler
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()


            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm progress bar
            progress_bar.set_postfix(loss=total_loss / total, accuracy=100 * correct / total)

        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Reduce LR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset, DataLoader
import numpy as np


# Number of folds
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Assuming dataset is a PyTorch Dataset object
dataset = train_loader.dataset  # Extract dataset from DataLoader
X = np.arange(len(dataset))  # Indices of samples
y = np.array([dataset[i][1] for i in range(len(dataset))])  # Labels

# Loop over folds
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"Fold {fold + 1}:")

    # Create train and validation subsets
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    # Create DataLoaders
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

    # Train model on this fold
    print(f"Training on Fold {fold + 1}...")
    train_model(model, train_loader, criterion, optimizer, num_epochs=50)


<ipython-input-36-be5b0441edac>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision Training


Fold 1:
Training on Fold 1...


Epoch 1/50:   0%|          | 0/254 [00:00<?, ?it/s]<ipython-input-36-be5b0441edac>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/50: 100%|██████████| 254/254 [01:21<00:00,  3.12it/s, accuracy=66.6, loss=0.031]


Epoch [1/50], Loss: 251.2646, Accuracy: 66.64%


Epoch 2/50: 100%|██████████| 254/254 [01:21<00:00,  3.13it/s, accuracy=67.9, loss=0.0284]


...

Epoch 40/50: 100%|██████████| 254/254 [01:21<00:00,  3.13it/s, accuracy=97.1, loss=0.00259]


Epoch [40/50], Loss: 20.9924, Accuracy: 97.07%


Epoch 41/50: 100%|██████████| 254/254 [01:21<00:00,  3.13it/s, accuracy=96.7, loss=0.00301]


Epoch [41/50], Loss: 24.4147, Accuracy: 96.66%


Epoch 42/50:  12%|█▏        | 31/254 [00:10<01:11,  3.10it/s, accuracy=96.2, loss=0.00298]

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Function to Compute and Display Confusion Matrix as Percentages
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100  # Convert to percentages

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_percentage, annot=True, fmt=".2f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix (Percentage)")
    plt.show()

# Function to Print Precision, Recall, and F1-Score Table
def print_classification_report(y_true, y_pred, class_names):
    report = classification_report(y_true, y_pred, target_names=class_names)
    print("🔹 Classification Report:")
    print(report)


In [ ]:
from sklearn.preprocessing import label_binarize

# Function to Compute and Plot ROC Curve (Class-wise + Overall)
def plot_roc_curve(y_true, y_scores, num_classes):
    plt.figure(figsize=(8, 6))

    # One-hot encode the true labels
    y_true_bin = label_binarize(y_true, classes=np.arange(num_classes))

    # Compute micro-average ROC curve and AUC
    fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), y_scores.ravel())
    roc_auc_micro = auc(fpr_micro, tpr_micro)

    # Compute macro-average ROC curve and AUC
    all_fpr = np.unique(np.concatenate([roc_curve(y_true_bin[:, i], y_scores[:, i])[0] for i in range(num_classes)]))
    mean_tpr = np.zeros_like(all_fpr)

    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

        # Interpolating TPR at unique FPR values
        mean_tpr += np.interp(all_fpr, fpr, tpr)

    mean_tpr /= num_classes
    roc_auc_macro = auc(all_fpr, mean_tpr)

    # Plot macro and micro ROC curves
    plt.plot(all_fpr, mean_tpr, label=f"Macro Avg (AUC = {roc_auc_macro:.2f})", linestyle="--", linewidth=2)
    plt.plot(fpr_micro, tpr_micro, label=f"Micro Avg (AUC = {roc_auc_micro:.2f})", linestyle=":", linewidth=2)

    # Diagonal reference line
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve (Class-wise + Overall)")
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
# Updated Evaluation Function
def evaluate_model(model, test_loader, class_names):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1).cpu().numpy()  # Convert logits to probabilities
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs)

    # Convert lists to numpy arrays
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    all_probs = np.array(all_probs)

    # 📊 Confusion Matrix
    plot_confusion_matrix(all_labels, all_preds, class_names)

    # 🏆 Precision, Recall, and F1-Score Table
    print_classification_report(all_labels, all_preds, class_names)

    # 📈 ROC Curve
    plot_roc_curve(all_labels, all_probs, num_classes=len(class_names))

# Call Evaluation Function
evaluate_model(model, test_loader, train_dataset.class_names)
